In [1]:
from sqlalchemy import create_engine, text, inspect, func
import sqlite3

# ORM imports
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Float, Boolean
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base

# API
import requests
import json

# Data Science and Visualization
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import datetime as dt

In [2]:
#CSV import for crashes table
filepath = "../Resources/SGO-2021-01_Incident_Reports_ADS.csv"
df = pd.read_csv(filepath)
df.head()

,Report ID,Report Version,Reporting Entity,Report Type,Report Month,Report Year,Report Submission Date,VIN,VIN - Unknown,Serial Number,...,Investigating Officer Name,Inv. Officer Name - Unknown,Investigating Officer Phone,Inv. Officer Phone - Unknown,Investigating Officer Email,Inv. Officer Email - Unknown,Within ODD?,Within ODD? - CBI,Narrative,Narrative - CBI?
0,30270-7157,3,Waymo LLC,Update,NaN,NaN,AUG-2024,SADHW2S13M1,,NaN,...,NaN,,NaN,,NaN,,Yes,,Waymo is submitting a second update to the rep...,NaN
1,30270-8406,1,Waymo LLC,Monthly,7.0,2024.0,AUG-2024,SADHW2S1XR1,,NaN,...,NaN,,NaN,,NaN,,Yes,,"On July [XXX], 2024 at 4:58 AM PT a Waymo Auto...",NaN
2,30270-8405,1,Waymo LLC,Monthly,7.0,2024.0,AUG-2024,SADHW2S17R1,,NaN,...,NaN,,NaN,,NaN,,Yes,,"On July [XXX], 2024 at 11:29 AM MT a Waymo Aut...",NaN
3,30270-8404,1,Waymo LLC,Monthly,7.0,2024.0,AUG-2024,SADHW2S1XR1,,NaN,...,NaN,,NaN,,NaN,,Yes,,"On July [XXX], 2024 at 4:16 PM PT a Waymo Auto...",NaN
4,30270-8403,1,Waymo LLC,Monthly,7.0,2024.0,AUG-2024,SADHW2S19R1,,NaN,...,NaN,,NaN,,NaN,,Yes,,"On July [XXX], 2024 at 11:06 AM CST a Waymo Au...",NaN


In [3]:
df.columns

Index(['Report ID', 'Report Version', 'Reporting Entity', 'Report Type',
       'Report Month', 'Report Year', 'Report Submission Date', 'VIN',
       'VIN - Unknown', 'Serial Number',
       ...
       'Investigating Officer Name', 'Inv. Officer Name - Unknown',
       'Investigating Officer Phone', 'Inv. Officer Phone - Unknown',
       'Investigating Officer Email', 'Inv. Officer Email - Unknown',
       'Within ODD?', 'Within ODD? - CBI', 'Narrative', 'Narrative - CBI?'],
      dtype='object', length=137)

In [4]:
len(df.columns)

137

In [5]:
#First Set of Columns to remove
# columns_to_remove = ['Report ID', 'Report Version', 'Reporting Entity', 'Report Submission Date', 'VIN', 'VIN – Unknown', 'Serial Number',\
#                     'Make', 'Model', 'Model – Unknown', 'Model Year',	'Model Year – Unknown',	'Same Vehicle ID', 'Mileage', 'Mileage – Unknown',\
#                     'ADAS/ADS System Version', 'ADAS/ADS System Version – Unk', 'ADAS/ADS System Version CBI',  'ADAS/ADS Hardware Version',\
#                     'ADAS/ADS Hardware Version – Unk', 'ADAS/ADS Hardware Version CBI', 'ADAS/ADS Software Version', 'ADAS/ADS Software Version – Unk',\
#                     'ADAS/ADS Software Version CBI', 'Other Reporting Entities?', 'Other Reporting Entities? – Unk', 'Other Reporting Entities? – NA',\
#                     'Federal Regulatory Exemption?', 'Other Federal Reg. Exemption', 'Federal Reg. Exemption – Unk', 'Federal Reg. Exemption – No',\
#                     'State or Local Permit?', 'State or Local Permit', 'ADS Equipped?, Automation System Engaged?',	'Operating Entity',\
#                     'Operating Entity – Unknown', 'Source - Complaint/Claim', 'Source – Telematics', 'Source - Law Enforcement', 'Source - Field Report',\
#                     'Source – Testing', 'Source – Media', 'Source – Other', 'Source - Other Text', 'Notice Received Date', 'Same Incident ID', 'Address',\
#                     'Address – Unknown', 'Crash With', 'Property Damage?', 'CP Pre-Crash Movement', 'CP Any Air Bags Deployed?', 'CP Was Vehicle Towed?',\
#                     'CP Contact Area - Rear Left', 'CP Contact Area – Left', 'CP Contact Area - Front Left', 'CP Contact Area – Rear', 'CP Contact Area – Top',\
#                     'CP Contact Area – Front', 'CP Contact Area - Rear Right', 'CP Contact Area – Right', 'CP Contact Area - Front Right',\
#                     'CP Contact Area – Bottom', 'CP Contact Area – Unknown', 'SV Pre-Crash Movement',	'SV Any Air Bags Deployed?', 'SV Was Vehicle Towed?',\
#                     'SV Contact Area - Rear Left', 'SV Contact Area – Left', 'SV Contact Area - Front Left', 'SV Contact Area – Rear', 'SV Contact Area – Top',\
#                     'SV Contact Area – Front', 'SV Contact Area - Rear Right', 'SV Contact Area – Right', 'SV Contact Area - Front Right',\
#                     'SV Contact Area – Bottom', 'SV Contact Area – Unknown', 'Data Availability – EDR', 'Data Availability - Police Rpt',\
#                     'Data Availability – Telematics', 'Data Availability – Complaints', 'Data Availability – Video', 'Data Availability – Other',\
#                     'Data Availability - No Data', 'Data Availability – Unknown', 'Law Enforcement Investigating?', 'Investigating Agency',\
#                     'Investigating Agency – Unknown', 'Rep Ent Or Mfr Investigating?', 'Investigating Officer Name', 'Inv. Officer Name – Unknown',\
#                     'Investigating Officer Phone', 'Inv. Officer Phone – Unknown', 'Investigating Officer Email', 'Inv. Officer Email – Unknown', 'Within ODD?',\
#                     'Within ODD? – CBI', 'Narrative', 'Narrative - CBI?']

# df2 = df.drop(columns_to_remove, axis=1)



In [6]:
cols_to_drop = ['Report ID', 'Report Version', 'Report Month', 'Report Year','Reporting Entity','Report Submission Date',\
                'VIN', 'Model - Unknown', 'Model Year - Unknown','ADAS/ADS Hardware Version',\
                'Same Vehicle ID', 'VIN - Unknown', 'Serial Number','Driver / Operator Type',\
                'Model - Unknown', 'Model Year - Unknown','Same Vehicle ID',\
               'Report Type', 'Mileage - Unknown', 'ADAS/ADS System Version - Unk',\
                'ADAS/ADS System Version CBI', 'ADAS/ADS Hardware Version - Unk','Investigating Officer Name',\
                'Inv. Officer Name - Unknown', 'Investigating Officer Phone', 'Inv. Officer Phone - Unknown','Investigating Officer Email',\
                'Inv. Officer Email - Unknown','Within ODD?', 'Within ODD? - CBI', 'Narrative', 'Narrative - CBI?','ADAS/ADS Hardware Version CBI',\
               'ADAS/ADS Software Version', 'ADAS/ADS Software Version - Unk',\
               'ADAS/ADS Software Version CBI', 'Other Reporting Entities?',\
               'Other Reporting Entities? - Unk', 'Other Reporting Entities? - NA',\
               'Federal Regulatory Exemption?','State or Local Permit?', 'State or Local Permit',\
               'Data Availability - Telematics', 'Data Availability - Complaints',\
               'Data Availability - Video', 'Data Availability - Other',\
               'Data Availability - No Data', 'Data Availability - Unknown',\
               'Law Enforcement Investigating?', 'Investigating Agency',\
               'Investigating Agency - Unknown', 'Rep Ent Or Mfr Investigating?','Other Federal Reg. Exemption',\
               'Federal Reg. Exemption - Unk', 'Federal Reg. Exemption - No',\
               'Automation System Engaged?', 'Source - Complaint/Claim',\
               'Source - Telematics', 'Source - Law Enforcement', 'ADAS/ADS System Version',\
               'Source - Field Report', 'Source - Testing', 'Source - Media',\
               'Source - Other', 'Source - Other Text','Incident Date - Unknown', 'Notice Received Date',\
               'Same Incident ID','Address', 'Address - Unknown','Weather - Other', 'Weather - Other Text',\
               'CP Pre-Crash Movement', 'CP Any Air Bags Deployed?','CP Was Vehicle Towed?', 'CP Contact Area - Rear Left',\
               'CP Contact Area - Left', 'CP Contact Area - Front Left','CP Contact Area - Rear', 'CP Contact Area - Top',\
               'CP Contact Area - Front', 'CP Contact Area - Rear Right','CP Contact Area - Right', 'CP Contact Area - Front Right',\
               'CP Contact Area - Bottom', 'CP Contact Area - Unknown',\
               'SV Pre-crash Speed - Unknown','SV Contact Area - Rear Left',\
                'SV Contact Area - Left','SV Contact Area - Front Left', 'SV Contact Area - Rear','SV Contact Area - Top',\
                'SV Contact Area - Front','SV Contact Area - Rear Right', 'SV Contact Area - Right','SV Contact Area - Front Right',\
                'SV Contact Area - Bottom','SV Contact Area - Unknown', 'Data Availability - EDR','Data Availability - Police Rpt',\
               'Operating Entity','Operating Entity - Unknown','Crash With', 'Weather - Unknown','Incident Time - Unknown', 'Latitude',\
                'Latitude - Unknown','Longitude', 'Longitude - Unknown','City - Unknown','Zip Code','Zip Code - Unknown',\
                'Posted Speed Limit - Unknown']
df2 = df.drop(cols_to_drop, axis=1)
df2.head()

,Make,Model,Model Year,Mileage,ADS Equipped?,Incident Date,Incident Time (24:00),City,State,Roadway Type,...,Weather - Fog/Smoke,Weather - Rain,Weather - Severe Wind,Highest Injury Severity Alleged,Property Damage?,SV Pre-Crash Movement,SV Any Air Bags Deployed?,SV Was Vehicle Towed?,SV Were All Passengers Belted?,SV Precrash Speed (MPH)
0,Jaguar,I-Pace,2021.0,56344.0,Yes,JAN-2024,12:04,Los Angeles,CA,Street,...,,,,Unknown,Yes,Stopped,No,No,"No, see Narrative",0.0
1,Jaguar,I-Pace,2024.0,7345.0,Yes,JUL-2024,04:58,San Francisco,CA,Street,...,,,,No Injuries Reported,Yes,Parked,No,No,No Passengers in Vehicle,0.0
2,Jaguar,I-Pace,2024.0,25984.0,Yes,JUL-2024,11:29,Phoenix,AZ,Parking Lot,...,,,,No Injuries Reported,Yes,Stopped,No,No,Yes,0.0
3,Jaguar,I-Pace,2024.0,449.0,Yes,JUL-2024,18:16,San Francisco,CA,Street,...,,,,No Injuries Reported,Yes,Stopped,No,No,No Passengers in Vehicle,0.0
4,Jaguar,I-Pace,2024.0,4329.0,Yes,JUL-2024,11:06,Austin,TX,Street,...,,,,No Injuries Reported,Yes,Stopped,No,No,Yes,0.0


In [7]:
# Refreshed look at columns
df2.columns

Index(['Make', 'Model', 'Model Year', 'Mileage', 'ADS Equipped?',
       'Incident Date', 'Incident Time (24:00)', 'City', 'State',
       'Roadway Type', 'Roadway Surface', 'Roadway Description',
       'Posted Speed Limit (MPH)', 'Lighting', 'Weather - Clear',
       'Weather - Snow', 'Weather - Cloudy', 'Weather - Fog/Smoke',
       'Weather - Rain', 'Weather - Severe Wind',
       'Highest Injury Severity Alleged', 'Property Damage?',
       'SV Pre-Crash Movement', 'SV Any Air Bags Deployed?',
       'SV Was Vehicle Towed?', 'SV Were All Passengers Belted?',
       'SV Precrash Speed (MPH)'],
      dtype='object')

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1362 entries, 0 to 1361
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Make                             1359 non-null   object 
 1   Model                            1358 non-null   object 
 2   Model Year                       1348 non-null   float64
 3   Mileage                          1322 non-null   float64
 4   ADS Equipped?                    1358 non-null   object 
 5   Incident Date                    1358 non-null   object 
 6   Incident Time (24:00)            1352 non-null   object 
 7   City                             1356 non-null   object 
 8   State                            1359 non-null   object 
 9   Roadway Type                     1359 non-null   object 
 10  Roadway Surface                  1359 non-null   object 
 11  Roadway Description              1359 non-null   object 
 12  Posted Speed Limit (

In [9]:
df2.shape

(1362, 27)

In [10]:
df2 = df2.dropna(subset = ['Model Year', 'SV Precrash Speed (MPH)', 'Incident Time (24:00)', 'Posted Speed Limit (MPH)', 'City', 'ADS Equipped?', 'Mileage'])

In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1290 entries, 0 to 1359
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Make                             1290 non-null   object 
 1   Model                            1290 non-null   object 
 2   Model Year                       1290 non-null   float64
 3   Mileage                          1290 non-null   float64
 4   ADS Equipped?                    1290 non-null   object 
 5   Incident Date                    1290 non-null   object 
 6   Incident Time (24:00)            1290 non-null   object 
 7   City                             1290 non-null   object 
 8   State                            1290 non-null   object 
 9   Roadway Type                     1290 non-null   object 
 10  Roadway Surface                  1290 non-null   object 
 11  Roadway Description              1290 non-null   object 
 12  Posted Speed Limit (MPH) 

In [12]:
df2.value_counts('SV Precrash Speed (MPH)')

SV Precrash Speed (MPH)
0.0     563
1.0      79
2.0      64
13.0     41
8.0      41
3.0      41
10.0     35
7.0      34
5.0      33
6.0      29
4.0      28
9.0      27
15.0     21
18.0     17
12.0     17
22.0     14
19.0     14
25.0     13
11.0     13
35.0     12
14.0     12
24.0     12
21.0     11
23.0     11
34.0     10
65.0      9
17.0      7
62.0      7
64.0      6
50.0      6
51.0      6
56.0      6
58.0      5
53.0      5
55.0      5
61.0      4
27.0      4
20.0      4
44.0      4
59.0      4
16.0      3
63.0      2
40.0      2
45.0      2
67.0      2
33.0      1
32.0      1
30.0      1
29.0      1
26.0      1
Name: count, dtype: int64

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1290 entries, 0 to 1359
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Make                             1290 non-null   object 
 1   Model                            1290 non-null   object 
 2   Model Year                       1290 non-null   float64
 3   Mileage                          1290 non-null   float64
 4   ADS Equipped?                    1290 non-null   object 
 5   Incident Date                    1290 non-null   object 
 6   Incident Time (24:00)            1290 non-null   object 
 7   City                             1290 non-null   object 
 8   State                            1290 non-null   object 
 9   Roadway Type                     1290 non-null   object 
 10  Roadway Surface                  1290 non-null   object 
 11  Roadway Description              1290 non-null   object 
 12  Posted Speed Limit (MPH) 

In [14]:
# Replace 'apple' with 'orange' in column 'B'
df2.loc[df2['Weather - Clear'] == 'Y', 'Weather - Clear'] = 'Clear'
df2.loc[df2['Weather - Snow'] == 'Y', 'Weather - Snow'] = 'Snow'
df2.loc[df2['Weather - Cloudy'] == 'Y', 'Weather - Cloudy'] = 'Cloudy'
df2.loc[df2['Weather - Fog/Smoke'] == 'Y', 'Weather - Fog/Smoke'] = 'Fog/Smoke'
df2.loc[df2['Weather - Rain'] == 'Y', 'Weather - Rain'] = 'Rain'
df2.loc[df2['Weather - Severe Wind'] == 'Y', 'Weather - Severe Wind'] = 'Severe Wind'
print(df2.head)

<bound method NDFrame.head of         Make   Model  Model Year  Mileage ADS Equipped? Incident Date  \
0     Jaguar  I-Pace      2021.0  56344.0           Yes      JAN-2024   
1     Jaguar  I-Pace      2024.0   7345.0           Yes      JUL-2024   
2     Jaguar  I-Pace      2024.0  25984.0           Yes      JUL-2024   
3     Jaguar  I-Pace      2024.0    449.0           Yes      JUL-2024   
4     Jaguar  I-Pace      2024.0   4329.0           Yes      JUL-2024   
...      ...     ...         ...      ...           ...           ...   
1353    Ford  Fusion      2019.0  10722.0           Yes      JUL-2021   
1354  Jaguar  I-Pace      2021.0   1232.0           Yes      JUL-2021   
1355  Jaguar  I-Pace      2021.0   1232.0           Yes      JUL-2021   
1358  Jaguar  I-Pace      2021.0   1232.0           Yes      JUL-2021   
1359  Jaguar  I-Pace      2021.0   1232.0           Yes      JUL-2021   

     Incident Time (24:00)           City State Roadway Type  ...  \
0                    12:

In [15]:
# Merge 'First Name' and 'Last Name' into a new column 'Full Name'
df2['Weather'] = df2[['Weather - Clear', 'Weather - Snow', 'Weather - Cloudy', 'Weather - Fog/Smoke', 'Weather - Rain', 'Weather - Severe Wind']].agg(' '.join, axis=1)


In [16]:
df2.value_counts('Weather')

Weather
Clear                   1070
    Cloudy               156
        Rain              40
                          18
    Cloudy   Rain          4
  Snow                     1
Clear   Cloudy             1
Name: count, dtype: int64

In [17]:
df2.loc[df2['Weather'] == '           ', 'Weather'] = 'Clear'
df2.value_counts('Weather')

Weather
Clear                   1070
    Cloudy               156
        Rain              40
Clear                     18
    Cloudy   Rain          4
  Snow                     1
Clear   Cloudy             1
Name: count, dtype: int64

In [18]:
df2['Weather'] = df2['Weather'].str.strip()
df2.loc[df2['Weather'] == 'Clear   Cloudy', 'Weather'] = 'Clear'
df2.loc[df2['Weather'] == 'Cloudy   Rain', 'Weather'] = 'Rain'
df2.loc[df2['Weather'] == '            ', 'Weather'] = 'Clear'
df2['Weather'].fillna("Clear", inplace=True)
df2['Weather'] = df2['Weather'].str.strip()
df2.head()

,Make,Model,Model Year,Mileage,ADS Equipped?,Incident Date,Incident Time (24:00),City,State,Roadway Type,...,Weather - Rain,Weather - Severe Wind,Highest Injury Severity Alleged,Property Damage?,SV Pre-Crash Movement,SV Any Air Bags Deployed?,SV Was Vehicle Towed?,SV Were All Passengers Belted?,SV Precrash Speed (MPH),Weather
0,Jaguar,I-Pace,2021.0,56344.0,Yes,JAN-2024,12:04,Los Angeles,CA,Street,...,,,Unknown,Yes,Stopped,No,No,"No, see Narrative",0.0,Clear
1,Jaguar,I-Pace,2024.0,7345.0,Yes,JUL-2024,04:58,San Francisco,CA,Street,...,,,No Injuries Reported,Yes,Parked,No,No,No Passengers in Vehicle,0.0,Clear
2,Jaguar,I-Pace,2024.0,25984.0,Yes,JUL-2024,11:29,Phoenix,AZ,Parking Lot,...,,,No Injuries Reported,Yes,Stopped,No,No,Yes,0.0,Clear
3,Jaguar,I-Pace,2024.0,449.0,Yes,JUL-2024,18:16,San Francisco,CA,Street,...,,,No Injuries Reported,Yes,Stopped,No,No,No Passengers in Vehicle,0.0,Clear
4,Jaguar,I-Pace,2024.0,4329.0,Yes,JUL-2024,11:06,Austin,TX,Street,...,,,No Injuries Reported,Yes,Stopped,No,No,Yes,0.0,Cloudy


In [19]:
df2.value_counts('Weather')


Weather
Clear     1089
Cloudy     156
Rain        44
Snow         1
Name: count, dtype: int64

In [20]:
df2 = df2.dropna(subset = 'Weather')

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1290 entries, 0 to 1359
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Make                             1290 non-null   object 
 1   Model                            1290 non-null   object 
 2   Model Year                       1290 non-null   float64
 3   Mileage                          1290 non-null   float64
 4   ADS Equipped?                    1290 non-null   object 
 5   Incident Date                    1290 non-null   object 
 6   Incident Time (24:00)            1290 non-null   object 
 7   City                             1290 non-null   object 
 8   State                            1290 non-null   object 
 9   Roadway Type                     1290 non-null   object 
 10  Roadway Surface                  1290 non-null   object 
 11  Roadway Description              1290 non-null   object 
 12  Posted Speed Limit (MPH) 

In [22]:
df2.value_counts('Model')

Model
I-Pace              581
AV                  378
Highlander           73
Escape               34
Pacifica             21
579 Epiq             19
Fusion               16
I-PACE               14
RX450H               13
579                  12
Sienna               12
T680                 11
Bolt                 10
Ioniq 5               8
Prius                 7
GEN2                  6
Air Touring           5
Cascadia              5
Kona EV               5
I- Pace               4
ID.Buzz               4
Leaf                  4
MKZ                   3
RX 450h               3
S450                  3
LEAF                  3
CASCADIA              3
EZ10 Gen 3            3
IONIQ 5               3
SPORTAGE              2
RX450h                2
579m                  2
Grand Cherokee        2
Camry                 2
Origin                2
Siennaa               1
Peterbilt 579         1
111324                1
Olli 2.0 Shuttle      1
Olli 1.0              1
Olli                  1
Leaf SV Pl

In [22]:
df2.value_counts('Make')

Make
Jaguar                     599
Cruise                     380
Toyota                      96
Ford                        50
Peterbilt                   34
Chrysler                    21
Lexus                       17
Hyundai                     17
Chevrolet                   10
Nissan                       8
Kenworth Motor Truck Co      8
EZ10                         6
FREIGHTLINER                 5
Lucid                        5
Volkswagen                   4
Local Motors                 3
Freightliner                 3
Lincoln                      3
Mercedes-Benz                3
KIA                          2
Kenworth                     2
Ligier                       2
Jeep                         2
NISSAN                       1
Crui9se                      1
Navya                        1
International                1
Local Motors (LOCM)          1
Ligier Group                 1
LExus                        1
INTERNATIONAL                1
KENWORTH                     1
Zoo

In [23]:
df2.value_counts('Model Year')

Model Year
2021.0    546
2023.0    265
2020.0    130
2022.0    116
2024.0     95
2016.0     45
2019.0     44
2017.0     41
2018.0      8
Name: count, dtype: int64

In [24]:
df2.value_counts('Mileage')

Mileage
5091.0       8
19560.0      6
2798.0       6
1111.0       6
3183.0       5
            ..
27380.0      1
27052.0      1
26823.0      1
26717.0      1
1150344.0    1
Name: count, Length: 713, dtype: int64

In [25]:
cols = ['Incident Date', 'Incident Time (24:00)','City',  'State','Roadway Type', 'Roadway Surface',\
        'Roadway Description', 'Posted Speed Limit (MPH)', 'Lighting', 'Weather - Clear','Weather - Snow', 'Weather - Cloudy', 'Weather - Fog/Smoke',\
        'Weather - Rain', 'Weather - Severe Wind', 'Weather - Other', 'Weather - Other Text','Highest Injury Severity Alleged',\
        'SV Were All Passengers Belted?']

for col in cols:
  print(col)
  print(df2[col].nunique())
  print(df2[col].value_counts())
  print()

Incident Date
38
Incident Date
AUG-2023    94
SEP-2023    56
JUL-2023    55
OCT-2023    54
MAR-2023    53
MAY-2024    52
JUN-2024    49
JUL-2024    48
AUG-2022    45
JUN-2023    45
MAY-2023    43
APR-2023    39
DEC-2022    39
APR-2022    36
JUL-2022    34
MAR-2022    33
NOV-2021    33
MAR-2024    32
JUN-2022    32
FEB-2023    31
FEB-2024    31
MAY-2022    29
OCT-2021    29
JAN-2022    26
SEP-2022    26
JAN-2024    26
APR-2024    25
DEC-2023    24
AUG-2021    24
SEP-2021    24
FEB-2022    21
JUL-2021    19
DEC-2021    18
NOV-2022    16
NOV-2023    15
OCT-2022    13
JAN-2023    12
AUG-2024     9
Name: count, dtype: int64

Incident Time (24:00)
550
Incident Time (24:00)
03:15    11
00:39    10
21:29    10
22:22    10
23:30     9
         ..
17:09     1
16:32     1
23:48     1
17:50     1
09:50     1
Name: count, Length: 550, dtype: int64

City
75
City
San Francisco    719
Phoenix          100
Tempe             80
Austin            73
Los Angeles       32
                ... 
BUFFALO      

KeyError: 'Weather - Other'

In [ ]:
df2['Incident Date'].value_counts()

In [26]:
df2['Incident Time (24:00)'].value_counts()

Incident Time (24:00)
03:15    11
00:39    10
21:29    10
22:22    10
23:30     9
         ..
17:09     1
16:32     1
23:48     1
17:50     1
09:50     1
Name: count, Length: 550, dtype: int64

In [27]:
df2['City'].value_counts()

City
San Francisco    719
Phoenix          100
Tempe             80
Austin            73
Los Angeles       32
                ... 
BUFFALO            1
Sunol              1
Emeryville         1
Barstow            1
Milpitas           1
Name: count, Length: 75, dtype: int64

In [28]:
df2['State'].value_counts()

State
CA     810
AZ     236
TX     125
NV      33
FL      32
MI      11
DC       8
MN       6
NM       6
CO       6
RI       5
IN       4
WA       3
PA       2
WY       2
GA       1
Name: count, dtype: int64

In [29]:
df2['Roadway Type'].value_counts()

Roadway Type
Intersection         574
Street               558
Highway / Freeway     93
Parking Lot           58
Traffic Circle         5
Unknown                2
Name: count, dtype: int64

In [30]:
df2['Roadway Surface'].value_counts()

Roadway Surface
Dry                   1223
Wet                     61
Unknown                  4
Snow / Slush / Ice       2
Name: count, dtype: int64

In [31]:
df2['Roadway Description'].value_counts()

Roadway Description
No Unusual Conditions    1230
Other, see Narrative       32
Work Zone                  19
Traffic Incident            7
Unknown                     2
Name: count, dtype: int64

In [32]:
df2['Posted Speed Limit (MPH)'].value_counts()

Posted Speed Limit (MPH)
25.0    730
30.0    146
35.0    105
45.0     58
15.0     53
65.0     51
20.0     45
40.0     39
75.0     27
5.0      11
10.0      5
55.0      5
60.0      4
70.0      3
0.0       3
9.0       3
50.0      1
1.0       1
Name: count, dtype: int64

In [33]:
df2['Lighting'].value_counts()

Lighting
Daylight                   738
Dark - Lighted             480
Dawn / Dusk                 44
Dark - Not Lighted          25
Dark - Unknown Lighting      3
Name: count, dtype: int64

In [34]:
df2['Weather - Clear'].value_counts()

Weather - Clear
Clear    1071
          219
Name: count, dtype: int64

In [35]:
df2['Weather - Snow'].value_counts()

Weather - Snow
        1289
Snow       1
Name: count, dtype: int64

In [36]:
df2['Weather - Cloudy'].value_counts()

Weather - Cloudy
          1129
Cloudy     161
Name: count, dtype: int64

In [37]:
df2['Weather - Fog/Smoke'].value_counts()

Weather - Fog/Smoke
     1290
Name: count, dtype: int64

In [38]:
df2['Weather - Rain'].value_counts()

Weather - Rain
        1246
Rain      44
Name: count, dtype: int64

In [39]:
df2['Weather - Severe Wind'].value_counts()

Weather - Severe Wind
     1290
Name: count, dtype: int64

In [40]:
df2['Highest Injury Severity Alleged'].value_counts()

Highest Injury Severity Alleged
No Injuries Reported    1036
Minor                    125
Unknown                   70
Moderate                  37
Serious                   20
Fatality                   2
Name: count, dtype: int64

In [41]:
df2['SV Were All Passengers Belted?'].value_counts()

SV Were All Passengers Belted?
Yes                         829
No Passengers in Vehicle    428
No, see Narrative            30
Unknown                       3
Name: count, dtype: int64

In [42]:
df2['State'].value_counts()

State
CA     810
AZ     236
TX     125
NV      33
FL      32
MI      11
DC       8
MN       6
NM       6
CO       6
RI       5
IN       4
WA       3
PA       2
WY       2
GA       1
Name: count, dtype: int64

In [23]:
df2.to_csv('Car_Accidents_Visualizations.csv', index=False)